In [27]:
import os

In [28]:
%pwd

'/Users/kevin/Desktop/Pokemon/Pikachu-or-Raichu--Project'

In [16]:
os.chdir("../")

In [29]:
%pwd

'/Users/kevin/Desktop/Pokemon/Pikachu-or-Raichu--Project'

In [30]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    local_data_file: Path
    unzip_dir: Path


In [31]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [32]:
class ConfigurationManager:
    def __init__(self):
        config_filepath = CONFIG_FILE_PATH
        params_filepath = PARAMS_FILE_PATH

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        # Ensure the directories exist
        create_directories([config.root_dir])
        
        # Create a DataIngestionConfig without source_URL
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config


In [33]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [34]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def check_local_file(self):
        """
        Check if the local file exists and log its size.
        """
        if os.path.exists(self.config.local_data_file):
            logger.info(f"Local file found at: {self.config.local_data_file} of size: {get_size(Path(self.config.local_data_file))}")
        else:
            logger.error(f"Local file not found at: {self.config.local_data_file}")
            raise FileNotFoundError(f"Local file not found at {self.config.local_data_file}")

    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted files to: {unzip_path}")


In [35]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    
    data_ingestion.check_local_file()  # Check if the file exists locally
    data_ingestion.extract_zip_file()  # Extract the file
except Exception as e:
    raise e


[2024-09-21 18:39:02,712: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-09-21 18:39:02,715: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-21 18:39:02,716: INFO: common: Created directory at: artifacts]
[2024-09-21 18:39:02,717: INFO: common: Created directory at: artifacts/data_ingestion]
[2024-09-21 18:39:02,717: INFO: 3737507069: Local file found at: /Users/kevin/Downloads/Pokemon.zip of size: 42968 KB]
[2024-09-21 18:39:03,120: INFO: 3737507069: Extracted files to: artifacts/data_ingestion]
